In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers

In [6]:
PINECONE_API_KEY = "05ab418b-9272-4645-b940-6d5681e195e4"
PINECONE_API_ENV= "gcp-starter"

In [7]:
#Extract data from PDF
def load_pdf(data):
    loader = DirectoryLoader(data,  
                    glob ="*.pdf",
                    )
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf("Data/")

TypeError: DirectoryLoader.__init__() got an unexpected keyword argument 'Loader_cls'